##### Открываем файл, проверяем, нет ли пустых строк

In [37]:
import json

twi_archive = []
for line in open('hw_3_twitter.json', 'r', encoding = 'utf-8'):
    if len(line) >1:
        twi_archive.append(json.loads(line))

##### 1. Считаем количество твитов

Вариант 1. Можно проверить, не попало ли что-то со значением None

In [398]:
a = 0

for twi in twi_archive:
    if twi is not None:
        a += 1

print('Количество твитов:', a)

Количество твитов: 2556


Вариант 2. Но в подготовке мы уже проверили, так что можно обойтись просто:

In [402]:
a_1 = len(twi_archive)

print('Количество твитов:', a_1)

Количество твитов: 2556


##### 2. Считаем количество удаленных твитов

Вариант 1. Ищем удаленные твиты, вычитаем их из общего количества

! Вопрос: чем так отличается значение 'delete', что len твита с таким ключом считается меньше 0?

In [401]:
b = 0 

for twi in twi_archive:
    if len(twi) >1:
        b += 1

print('Количество удаленных твитов:', len(twi_archive)-b)

Количество удаленных твитов: 362


Вариант 2. Можно проверить каждый твит на удаленные объекты

In [404]:
c = 0 

for twi in twi_archive:
    for tweetkey in twi:
        if tweetkey == 'delete':
           c +=1

print('Количество удаленных твитов:', c)

Количество удаленных твитов: 362


##### 3. Какие языки твитов самые популярные? Топ-20

Импортируем каунтер, проверяем, есть ли в json информация о языке, если есть, то добавляем в каунтер. Делаем счетчик, чтобы посмотреть, какой язык на каком месте.

In [422]:
from collections import Counter

lang_dict = Counter()

for twi in twi_archive:
    if twi.get('lang') is not None:
        lang = twi['lang']
        lang_dict[lang] += 1
        
z = 1   
print('Самые популярные языки твитов:\n')
for lang,key in lang_dict.most_common(20):
    print(z, '-', lang, key)
    z += 1

Самые популярные языки твитов:

1 - en 719
2 - ja 438
3 - es 173
4 - ko 149
5 - th 123
6 - ar 119
7 - und 117
8 - in 71
9 - pt 69
10 - fr 35
11 - tr 30
12 - tl 29
13 - hi 23
14 - ru 15
15 - fa 8
16 - zh 8
17 - ca 7
18 - ur 6
19 - nl 6
20 - de 6


Вопрос: как здесь сделать красивее вывод? <br> 1) как избавиться от пробела в между элементами в print(a, '-')? <br> 2) как лучше сделать нумерацию элементов каунтера? не через доп. счетчик. есть ли что-то встроенное?

##### 4. Есть ли пользователи, у которых в набор попало несколько твитов? Сколько таких пользователей?

Проверяем, есть ли информация о пользователе. Лучше брать строку, потому что она обрабатывается быстрее, чем файл int. <br> Делаем словарь, в который записываем каждого пользователя. Если в наборе id пользователя встречается хотя бы 2 раза, он уже подходит под наше условие, мы добавляем его в счетчик, и дальнейшие вхождения в набор нам не интересны.

In [407]:
id_dict = Counter()
e = 0

for twi in twi_archive:
    if twi.get('id_str') is not None:
        ids = twi['id_str']
        id_dict[ids] += 1
        if id_dict[ids] == 2:
            e += 1
print('Количество пользователей, чьи твиты встречаются в наборе несколько раз:', e)        

Количество пользователей, чьи твиты встречаются в наборе несколько раз: 3


##### 5. Топ-20 хэштегов

Создаем словарь. Проверяем на наличие entities, потому что именно они содержат информацию о хэштегах. Проверяем наличие хэштегов, если есть, добавляем в словарь. Выводим хэштеги (количество и номер в топе тоже, чтобы было нагляднее).

In [415]:
tag_dict = Counter()

for twi in twi_archive:
    if twi.get('entities') is not None:
        if twi['entities'].get('hashtags') is not None:
            tags = twi['entities']['hashtags']
            for tag in tags:
                hashtag = tag['text']
                tag_dict[hashtag] += 1
                
print('Самые популярные хэштеги:\n')
z = 1             
for tag_dict, key in tag_dict.most_common(20):
    print(z, '-', tag_dict, key)
    z += 1

Самые популярные хэштеги:

1 - BTS 17
2 - 방탄소년단 13
3 - AMAs 11
4 - 人気投票ガチャ 8
5 - 태형 7
6 - 뷔 6
7 - BTSinChicago 5
8 - BTSLoveYourselfTour 5
9 - 오늘의방탄 5
10 - PledgeForSwachhBharat 5
11 - MPN 5
12 - PCAs 4
13 - V 4
14 - 시카고1회차공연 4
15 - เป๊กผลิตโชค 4
16 - JIMIN 4
17 - running 3
18 - NCT 3
19 - 지민 3
20 - WajahmuPlastik 3


##### 6. Составляем частотный словарь лексики из оригинальных твитов на английском языке

In [417]:
from string import punctuation # Импортируем из модуля пунктуацию. 
punct = punctuation + '\u2026' + '「」『』' # В ней есть не все специальные знаки набора, добавляем те, что тоже встречаются.

import re # Импортируем регулярные выражения

en_dictionary = Counter()

for twi in twi_archive:
    if twi.get('lang') is not None: # Проверяем язык, убираем из набора ретвиты и цитаты.
        if twi['lang'] == 'en':
            if twi.get('retweeted_status') is None and twi.get('quoted_status') is None:
                if twi.get('extended_tweet') is not None:
                    txt = twi['extended_tweet'].get('full_text') # Если встречаются большие твиты с медиа-информацией, обращаемся к объекту 'full_text'. 
                else:
                    txt = twi.get('text') #  Из твитов без медиа достаем информацию из 'text'.
                t = re.sub('https://t.co/.+', '', txt, flags=re.M) # Чистим полученные строки от ссылок; нижние подчеркивания, которыми разделяют слова, заменяем на пробелы. Порой одна фраза отображается на нескольких строках, поэтому лучше поставить флаг Multiline, чтобы избежать ошибок.
                tt = re.sub('_', ' ', t, flags=re.M)
                for word in tt.lower().split(): # Разбиваем слова и попутно приводим к одному регистру.
                    no_dots = re.sub('\.+?', ' ', word) # Удаляем все возможные точки между словами без пробелов (burning...man), которые может пропустить метод strip.
                    clean_word = no_dots.strip(punct) # Чистим от пунктуации и дополнительных знаков (переменная punct).
                    if clean_word:
                        en_dictionary[clean_word] += 1 # Очищенное слово добавляем в словарь, печатаем словарь.

print ('Частотный словарь: \n')
z = 1             
for en_dictionary, key in en_dictionary.most_common(50):
    print(z, '\t', en_dictionary, '\t', key)
    z += 1

Частотный словарь: 

1 	 the 	 133
2 	 to 	 95
3 	 a 	 74
4 	 and 	 68
5 	 i 	 66
6 	 you 	 51
7 	 is 	 50
8 	 of 	 50
9 	 for 	 44
10 	 it 	 44
11 	 in 	 42
12 	 that 	 34
13 	 on 	 29
14 	 me 	 28
15 	 my 	 26
16 	 be 	 23
17 	 are 	 22
18 	 your 	 21
19 	 this 	 21
20 	 but 	 21
21 	 with 	 21
22 	 so 	 20
23 	 more 	 19
24 	 have 	 19
25 	 get 	 18
26 	 not 	 18
27 	 at 	 18
28 	 just 	 16
29 	 about 	 15
30 	 what 	 15
31 	 from 	 15
32 	 all 	 13
33 	 out 	 13
34 	 was 	 12
35 	 they 	 12
36 	 one 	 12
37 	 we 	 12
38 	 now 	 11
39 	 only 	 11
40 	 or 	 10
41 	 amp 	 10
42 	 today 	 10
43 	 up 	 10
44 	 like 	 10
45 	 hit 	 10
46 	 he 	 10
47 	 it's 	 10
48 	 can 	 9
49 	 an 	 9
50 	 by 	 9


##### 7. Топ-10 авторов набора

Проверяем, содержит ли json информацию о пользователе и его подписчиках, делаем частотный словарь.

In [413]:
top = Counter()

for twi in twi_archive:
    if twi.get('user') is not None:
        name = twi['user'].get('screen_name')
        followers = twi['user'].get('followers_count')
        top[name] = followers

print('Самые популярные блоги: \n')
for name, key in top.most_common(10):
    print('@' + name + '|' + str(top[name]))

Самые популярные блоги: 

@FIL0S0FIA|2521403
@FitnessMagazine|1491309
@malaysiakini|1206759
@NYTScience|1137374
@GramaticaReal|625463
@tgrthabertv|392472
@TheSunFootball|383698
@Melbourne|374222
@Roznama_Express|318189
@burger_boogie|311319


##### 8. Топ-10 источников твитов

Проверяем, есть ли источник у твита. >Информация о клиенте заключена в< ненужное не включаем в группу. На всякий случай добавляем флаги. Добавляем клиенты в словарь, печатаем словарь.

In [419]:
source = Counter()

for twi in twi_archive:
    if twi.get('source') is not None:
        string = (twi['source'])
        client = re.match('(?:.+?>)(.+)?(?:<.+?)', string, flags=re.M|re.DOTALL)
        web_client = client.group(1)
        source[web_client] += 1

print ('Самые популярные клиенты для твиттера в наборе:\n')
z = 1
for source, key in source.most_common(10):
    print(z, '\t', source)
    z += 1

Самые популярные клиенты для твиттера в наборе:

1 	 Twitter for iPhone
2 	 Twitter for Android
3 	 Twitter Web Client
4 	 twittbot.net
5 	 Twitter Lite
6 	 Twitter for iPad
7 	 TweetDeck
8 	 Facebook
9 	 IFTTT
10 	 تطبيق قرآني
